# Automate Your Data Analysis Testing

PyCon Canada 2016 - Stephen Childs

About Me

* economics training
* data-driven economics policy research
* Institutional Research - student data reporting

* heard about testing, TDD
* wanted to apply that to a real world project
* a lot of my work involves preparing data for analysis
* wrote a test suite for my project
* the test took a **long** time to run - longer than this talk

## Agenda

* Introduction
* `Py.test`
* `NumPy.testing`
* Unit tests
* Testing your data
* Comparing Data Frames

## `Py.test`

[http://doc.pytest.org/en/latest/](http://doc.pytest.org/en/latest/)

* Major Python test framework.
* Runs `unittest` and `nose` tests as well.
* Re-writes assert statements - [http://doc.pytest.org/en/latest/assert.html#assert](http://doc.pytest.org/en/latest/assert.html#assert)
* Fixtures - `@pytest.fixture` decorator - [http://doc.pytest.org/en/latest/fixture.html#fixture](http://doc.pytest.org/en/latest/fixture.html#fixture)

### Fixture Example

In [ ]:
@pytest.fixture(scope='module')
def output_data():
    from project.db_engines import create_mssql_engine
    engine = create_mssql_engine()
    return pd.read_sql_table(‘output_data', engine)

You define the fixture as a function - pass it a `scope` parameter to determine how long it will be retained. Then you specify the name of the fixture into your test function.

In [ ]:
def test_unique_student(output_data):
    df = output_data
    assert True not in df.duplicated(subset=['id’]).unique()

### NumPy Testing

A whole set of tools is avaiable for testing NumPy arrays. The documentation is at [https://docs.scipy.org/doc/numpy/reference/routines.testing.html](https://docs.scipy.org/doc/numpy/reference/routines.testing.html)

* These work well with `py.test`
* They take `NumPy` arrays, not `pandas` Data Frames.
* use `Series.values` and `DataFrame.values` to convert.

* `np.testing.assert_array_equal()`
* `np.testing.assert_array_close()`

### Unit Tests

* These tools let you write unit tests
* Break data preparation code into small functions
* Think of a test for all cases
*Supply fake data to cover all tests

## Creating Test Dataset

This next part of the workbook will create our play dataset. It is random numbers that are generated with columns that are the names of TTC subways stations.

I generate one dataset (`x`) and modify it a bit to produce `y`.

I create a `RandomState` to get the same data each time.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import gendata

In [3]:
prng = np.random.RandomState(1234567890)

In [4]:
cols = gendata.get_cols(prng)
x = gendata.original_data(cols, prng)
y = gendata.modified_data(x, cols, prng)

Create a subset of the data to make it managable.

In [5]:
x1 = x.iloc[0:5,0:5]
y1 = y.iloc[0:5,0:5]

In [6]:
x1

,Finch,Davisville,Pape,Sheppard-Yonge,Donlands
0,24,2,18,24,12
1,9,20,1,7,21
2,22,2,23,22,4
3,15,20,7,16,9
4,6,3,12,9,16


In [7]:
y1

,Finch,Pape,Sheppard-Yonge,Donlands,Keele
0,24,20,24,12,18
1,9,1,7,21,1
3,15,7,16,9,24
4,6,12,9,16,12
5,7,6,21,16,20


### Testing Your Output Data

* Run statistics (`describe` method) – are they what you expect.
* Are the observations unique
* Columns of the expected type
* Observations in the expected range


In [7]:
x.iloc[:,:5].describe()

,Finch,Davisville,Pape,Sheppard-Yonge,Donlands
count,100.000000,100.00000,100.000000,100.000000,100.000000
mean,12.710000,12.04000,10.350000,12.460000,11.000000
std,6.986705,6.74038,6.662688,6.887875,6.553586
min,1.000000,1.00000,1.000000,1.000000,1.000000
25%,7.000000,6.00000,5.000000,7.000000,6.000000
50%,14.000000,13.00000,9.000000,12.000000,10.000000
75%,19.000000,18.00000,15.000000,19.000000,16.000000
max,24.000000,24.00000,24.000000,24.000000,24.000000


### Comparing Data Frames

We now have two dataframes with small differences. We want an automated way to look at those differences.

* Compare output of your code to previous versions.
* Store versions in database – datetime stamp and “reliability” flag.
* Compare against the last solid version.
* Useful for legacy (no unit tests) code.

#### Implementing Data Frame Comparison

* Relies on knowing your data
* Have variables that define rows and columns uniquely
* Put those unique identifiers in the index
* `set_index()` and `reset_index()`

We start with looking at rows that have been removed

In [ ]:
print('Rows removed: {}', np.setdiff1d(x.index, y.index))

In [8]:
x1.loc[np.setdiff1d(x1.index,y1.index)]

,Finch,Davisville,Pape,Sheppard-Yonge,Donlands
2,22,2,23,22,4


Then we can look at rows that have been added.

In [ ]:
print('Rows removed: {}', np.setdiff1d(y.index, x.index))

In [9]:
y1.loc[np.setdiff1d(y1.index,x1.index)]

,Finch,Pape,Sheppard-Yonge,Donlands,Keele
5,7,6,21,16,20


Then we can do the same for columns. First - removed columns.

In [ ]:
print('Columns removed: {}', np.setdiff1d(x.columns, y.columns))

In [10]:
x1.loc[:,np.setdiff1d(x1.columns, y1.columns)]

,Davisville
0,2
1,20
2,2
3,20
4,3


And added columns

In [ ]:
print('Columns added: {}', np.setdiff1d(y.columns, x.columns))

In [11]:
y1.loc[:,np.setdiff1d(y1.columns, x1.columns)]

,Keele
0,18
1,1
3,24
4,12
5,20


Now that we can see what's been added and removed... we want to focus on the rows and columsn that are the same across the two data frames.

In [12]:
same_x1 = x1.loc[np.intersect1d(x1.index, y1.index),
                 np.intersect1d(x1.columns, y1.columns)]
same_y1 = y1.loc[np.intersect1d(x1.index, y1.index),
                 np.intersect1d(x1.columns, y1.columns)]

In [13]:
same_x1

,Donlands,Finch,Pape,Sheppard-Yonge
0,12,24,18,24
1,21,9,1,7
3,9,15,7,16
4,16,6,12,9


In [14]:
same_y1

,Donlands,Finch,Pape,Sheppard-Yonge
0,12,24,20,24
1,21,9,1,7
3,9,15,7,16
4,16,6,12,9


In [15]:
def compdf(x,y):
    return (x.loc[~((x == y).all(axis=1)),
                  ~((x == y).all(axis=0))][~(x==y)].applymap(str) +
            ' -> ' +
            y.loc[~((x == y).all(axis=1)),
                  ~((x == y).all(axis=0))][~(x==y)].applymap(str)
           ).replace('nan -> nan', ' ', regex=True)

In [16]:
compdf(same_x1, same_y1)

,Pape
0,18 -> 20


In [17]:
x.loc[np.setdiff1d(x.index,y.index)].iloc[:,:10]

,Finch,Davisville,Pape,Sheppard-Yonge,Donlands,Keele,Bathurst,St Clair,Eglinton West,Old Mill
2,22,2,23,22,4,12,2,14,17,2
21,11,13,5,8,1,11,13,9,23,7
30,21,10,24,8,6,12,4,14,4,15
81,4,5,1,23,6,19,20,2,10,16
86,24,4,5,12,9,10,2,9,3,5


In [18]:
y.loc[np.setdiff1d(y.index,x.index)].iloc[:10,:10]

,Finch,Pape,Sheppard-Yonge,Donlands,Keele,Bathurst,St Clair,Eglinton West,Old Mill,Ellesmere
101,9,5,14,16,6,8,6,18,1,20
102,16,23,1,1,7,22,4,16,8,21
103,1,7,2,21,9,14,13,20,8,24
104,2,20,2,24,1,3,8,21,18,11
105,19,10,15,17,15,7,23,23,3,14
106,10,12,17,15,20,5,24,14,7,23
107,21,7,16,5,2,7,8,19,18,18
108,14,10,1,5,22,5,15,11,16,21
109,7,21,5,6,11,7,22,19,14,3
110,12,8,15,20,11,15,23,1,16,7


In [19]:
x.loc[:,np.setdiff1d(x.columns, y.columns)].iloc[:5,:]

,Davisville,Greenwood,Queen,Woodbine
0,2,7,3,3
1,20,22,1,15
2,2,5,13,18
3,20,20,17,17
4,3,1,24,5


In [20]:
y.loc[:,np.setdiff1d(y.columns, x.columns)].iloc[:5,:10]

,Chester,Christie,High Park,Islington,Kennedy,King,Kipling,Lawrence East,Lawrence West,Museum
0,18,12,8,8,13,10,19,17,24,5
1,11,12,22,3,17,8,9,11,9,3
3,23,3,2,1,21,13,17,22,14,16
4,8,18,18,16,16,16,16,2,20,16
5,8,1,11,4,20,3,24,18,11,8


In [21]:
same_x = x.loc[np.intersect1d(x.index, y.index),
                 np.intersect1d(x.columns, y.columns)]
same_y = y.loc[np.intersect1d(x.index, y.index),
                 np.intersect1d(x.columns, y.columns)]

In [22]:
compdf(same_x, same_y)

,Bathurst,Dupont,Old Mill,Pape,Wilson
0,,,,18.0 -> 20.0,
29,,,,,12.0 -> 16.0
49,,,14.0 -> 21.0,,
50,,21.0 -> 26.0,,,
69,21.0 -> 19.0,,,,


### Handling Missing Values

The dataset above doesn't have missing values -- which not always the case.

Missing values are tricky because they don't equal each other.

In [34]:
e = np.array([1,2,3,np.nan,4])
f = np.array([1,2,3,np.nan,5])
g = np.array([np.nan,3,3,np.nan,4])

In [36]:
e_isnan, f_isnan, g_isnan = np.isnan(e), np.isnan(f), np.isnan(g)

In [39]:
e_isnan == f_isnan

array([ True,  True,  True,  True,  True], dtype=bool)

In [40]:
e[~e_isnan] == f[~f_isnan]

array([ True,  True,  True, False], dtype=bool)

In [41]:
e_isnan == g_isnan

array([False,  True,  True,  True,  True], dtype=bool)

### Automated Comparison

* Number of rows/columns added/removed - compare % to threshold number
* Number of differences within common rows/columns – more than expected?
* Differences clustered in 1 row or column


* Follow me on twitter: [@sechilds](https://twitter.com/sechilds)
* PyData Calgary – [www.meetup.com/PyData-Calgary/](https://www.meetup.com/PyData-Calgary/)
* PyYYC - [www.meetup.com/py-yyc/](https://www.meetup.com/py-yyc/)
